In [1]:
# Force reinstall all langchain packages to the latest matching versions
%pip install -U --force-reinstall langchain langchain-community langchain-core langchain-google-genai valyu prophet yfinance matplotlib pandas

  Using cached langchain-1.2.9-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached langchain_google_genai-4.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached valyu-2.5.4-py3-none-any.whl.metadata (22 kB)
  Using cached prophet-1.3.0-py3-none-macosx_11_0_arm64.whl.metadata (3.5 kB)
  Using cached yfinance-1.1.0-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached matplotlib-3.10.8-cp313-cp313-macosx_11_0_arm64.whl.metadata (52 kB)
  Using cached pandas-3.0.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (79 kB)
  Using cached langgraph-1.0.8-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.6.9-py3-none-any.whl.metadata (15 kB)
  Using cached packaging-26.0-py3-none-any.whl.metadata (3.3 kB)
  Using

In [ ]:
import os
import operator
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from typing import Annotated, Literal, TypedDict, List

# --- LIBRARIES ---
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langgraph.prebuilt import create_react_agent 
from pydantic import BaseModel, Field
from valyu import Valyu 
from langchain.messages import SystemMessage, HumanMessage





model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,  
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [ ]:
@tool
def mlModel(ticker: str):
    """ 
    Uses Facebook Prophet to predict stock prices 30 days ahead.
    Returns daily price targets and saves a plot to a .png file.
    the png files name is generated by graph_filename = f"{ticker}_forecast.png"
    """

    try:
        # search stock data

        data = yf.Ticker(ticker).history(period="2y")
        if data.empty:
            return "Error: No data found"
        
        # Format for prophet
        
        df = data.reset_index()
        df['ds'] = df['Date'].dt.tz_localize(None)
        df['y'] = df['Close']

        # Train

        m = Prophet(daily_seasonality=True)
        m.fit(df)

        # Predict

        future = m.make_future_dataframe(periods=30)
        forecast = m.predict(future)

        # Save the Graph

        fig1 = m.plot(forecast)
        plt.title(f"{ticker} 30-Day Forecast")
        plt.xlabel("Date")
        plt.ylabel("Price")

        graph_filename = f"{ticker}_forecast.png"
        plt.savefig(graph_filename)
        plt.close()

        # List 30 day prediction
        future_data = forecast.tail(30)

        daily_tracking = []
        for index, row in future_data.iterrows():
            date_str = row['ds'].strftime('%Y-%m-%d')
            price_str = f"${row['yhat']:.2f}"
            daily_tracking.append(f"{date_str}: {price_str}")

        daily_summary = "\n".join(daily_tracking)

        latest_pred = forecast.iloc[-1]['yhat']
        current_price = df.iloc[-1]['y']
        trend = "UP" if latest_pred > current_price else "DOWN"
        print(trend)
        
        # Return Everything
        return (f"Analysis Complete for {ticker}\n"
                f"Graph saved to: {graph_filename}\n"
                f"Trend: {trend}\n\n"
                f"Daily Price Targets (Next 30 Days):\n"
                f"{daily_summary}")
    
    except Exception as e:
        return f"Prediction failed: {e}"
    

@tool 
def brownianModel(TICKER: str):
    """ 
    Uses geometric brownian motion and monte carlo method to predict stock prices 30 days ahead, for a given ticker. 
    Returns daily price predictions 30 trading days ahead, with a historical lookback period of 24 months.
    This can only be used if the stock has at least a two-year old history. 
    """
    try:
        # main variables
        scen_size = 1000 # Reduced for speed, increase if needed
        HISTORICAL_YEARS = 2
        PREDICTION_DAYS = 30
        stock_name = TICKER

        end_date = pd.Timestamp.today().normalize()
        start_date = end_date - pd.DateOffset(years=HISTORICAL_YEARS)
        pred_end_date = end_date + pd.tseries.offsets.BDay(PREDICTION_DAYS)

        # -----------------------------
        # Download and prepare data
        # -----------------------------
        # Fix for yfinance returning multi-index
        prices = yf.download(tickers=stock_name, start=start_date, end=pred_end_date, progress=False)
        
        if prices.empty:
            return f"Error: No data found for {stock_name}"

        # Handle yfinance MultiIndex (recent API change)
        if isinstance(prices.columns, pd.MultiIndex):
            prices = prices['Close']
            # If still a dataframe (multiple tickers?), select the specific ticker
            if isinstance(prices, pd.DataFrame) and stock_name in prices.columns:
                 prices = prices[stock_name]
        elif 'Close' in prices.columns:
            prices = prices['Close']
        
        # Ensure we have a Series, not a DataFrame
        if isinstance(prices, pd.DataFrame):
             prices = prices.iloc[:, 0]

        # Generate business days (weekdays only)
        future_dates = pd.bdate_range(start=pd.to_datetime(end_date) + pd.Timedelta(days=1),
                        end=pd.to_datetime(pred_end_date))

        train_set = prices.loc[:end_date]
        
        if len(train_set) < 2:
            return "Error: Not enough historical data for prediction."

        daily_returns = ((train_set / train_set.shift(1)) - 1).dropna()

        So = train_set.iloc[-1]
        dt = 1  # day
        
        # Calculate volatility and drift
        mu = np.mean(daily_returns)
        sigma = np.std(daily_returns)
        
        # If sigma is 0 or NaN, we can't predict
        if np.isnan(sigma) or sigma == 0:
            return "Error: Volatility calculation failed (sigma is 0 or NaN)."

        # Simulation
        T_days = len(future_dates)
        N = T_days
        t = np.arange(1, N + 1)

        b = {str(scen): np.random.normal(0, 1, N) for scen in range(1, scen_size + 1)}
        W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}

        drift = (mu - 0.5 * sigma ** 2) * t
        diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

        S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
        
        # Average prediction
        S_pred = np.mean(S, axis=0)

        final_df = pd.DataFrame({
            'pred': S_pred
        }, index=future_dates[:len(S_pred)])

        # Create output string
        rows = [f"{date.date()}: ${price:.2f}" for date, price in zip(final_df.index, final_df['pred'])]
        result = '\n'.join(rows)
        print(result)

        if not result:
            return "Error: Model ran but produced no output rows."

        return f"Brownian Motion Forecast for {stock_name}:\n{result}"

    except Exception as e:
        # This prevents the 'ValueError: contents are required' crash
        return f"Brownian Model failed: {str(e)}"


@tool 
#query_general is the prompt given to Valyu, and it should be on general / industry specific news/articles instead of stoc specific.
#max_results refers to the number of sources that should be returned by the function. (Top N)
def generalInfo(query_general: str) -> str:
    """
    query_general: the query that is to be sent to the ai, to find information regarding the news and researches focused on the macroeconomy and broad industry-specific news and researches relating to the stock. The top 10 relevant information will be recorded. 
    news within the most recent 12 months will be considered.
    """
    
    # Read the API key from environment
    API_KEY = os.environ.get("VALYU_API_KEY")
    # Initialize the Valyu client
    valyu = Valyu(api_key=API_KEY)

    # ---------- NEWS SEARCH ----------
    news_response = valyu.search(
        query=query_general,
        search_type="news",
        max_num_results=10,
        relevance_threshold=0.7,
        max_price=0.0, #free content only
        start_date=datetime.now().date() - timedelta(days=365),
        end_date=datetime.now().date(),
        excluded_sources=["reddit.com", "twitter.com", "x.com"],
        response_length="medium",
        fast_mode=False,
    )

    # ---------- PROPRIETARY SEARCH ----------
    proprietary_response = valyu.search(
        query=query_general,
        search_type="proprietary",
        max_num_results=10,
        relevance_threshold=0.7,
        max_price=0.0,
        start_date=datetime.now().date() - timedelta(days=365),
        end_date=datetime.now().date(),
        response_length="medium",
        fast_mode=False,
    )

    # Combine results safely
    response = (
        (news_response.get("results") or []) +
        (proprietary_response.get("results") or [])
    )


    results_list = []

    for result in response.get('results', []):
        results_list.append({
            "title": result.get('title', 'No title'),
            "url": result.get('url', 'No URL'),
            "snippet": result.get('snippet') or result.get('content', 'No snippet')
        })

    #Turns result_list (a dictioanry) into a readable string.

    lines = []
    for r in enumerate(results_list, 1):
        lines.append(
            f"TITLE: {r['title']}\n"
            f"URL: {r['url']}\n"
            f"Summary: {r['snippet']}\n"
        )
    response_str = "\n".join(lines)
    return str(response_str)



@tool 
def specificInfo(query_specific: str) -> str:
    """
    the input will be labelled query_general, which is the query that is to be sent to the ai, to find information regarding the news and researches focused on data referring to the specific stock in question. The top 10 relevant information will be recorded. 
    news within the most recent 12 months will be considered.
    """

    # Read the API key from environment
    API_KEY = os.environ.get("VALYU_API_KEY")
    # Initialize the Valyu client
    valyu = Valyu(api_key=API_KEY)

    # ---------- NEWS SEARCH ----------
    news_response = valyu.search(
        query=query_specific,
        search_type="news",
        max_num_results=10,
        relevance_threshold=0.7,
        max_price=0.0, #free content only
        start_date=datetime.now().date() - timedelta(days=365),
        end_date=datetime.now().date(),
        excluded_sources=["reddit.com", "twitter.com", "x.com"],
        response_length="medium",
        fast_mode=False,
    )

    # ---------- PROPRIETARY SEARCH ----------
    proprietary_response = valyu.search(
        query=query_specific,
        search_type="proprietary",
        max_num_results=10,
        relevance_threshold=0.7,
        max_price=0.0,
        start_date=datetime.now().date() - timedelta(days=365),
        end_date=datetime.now().date(),
        response_length="medium",
        fast_mode=False,
    )

    # Combine results safely
    response = (
        (news_response.get("results") or []) +
        (proprietary_response.get("results") or [])
    )


    results_list = []

    for result in response.get('results', []):
        results_list.append({
            "title": result.get('title', 'No title'),
            "url": result.get('url', 'No URL'),
            "snippet": result.get('snippet') or result.get('content', 'No snippet')
        })

    #Turns result_list (a dictioanry) into a readable string.

    lines = []
    for r in enumerate(results_list, 1):
        lines.append(
            f"TITLE: {r['title']}\n"
            f"URL: {r['url']}\n"
            f"Summary: {r['snippet']}\n"
        )
    response_str = "\n".join(lines)
    return str(response_str)


In [ ]:
trend_prompt = "You are a Quantitative Analyst. Use the provided ML and Statistical tools to analyze the stock ticker provided. ONLY ENTER THE ABBREVIATION OF THE STOCK TO THE TOOLS. Summarize the technical outlook."
trend_agent = create_agent(model, system_prompt=SystemMessage(content=[{"type": "text", "text": trend_prompt}, {"type": "text", "text": "stock markets"}], ), tools=[mlModel, brownianModel])

noise_prompt = "You are a Market Researcher. Use the search tool to find recent news, sentiment, and macro factors affecting the stock."
noise_agent = create_agent(model, [valyu_search_tool], system_prompt=SystemMessage(content=[{"type": "text", "text": noise_prompt}, {"type": "text", "text": "stock markets"}], ))


In [ ]:
result = trend_agent.invoke({"messages": [HumanMessage("analyze AMZN stock")]})
print(result)


In [ ]:
from pydantic import BaseModel, Field
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END



class AgentState(TypedDict):
    query: str
    results: Annotated[List[str], operator.add] 

def run_trend_agent(state: AgentState):
    """Invokes the Quant Agent"""
    print("Executing Trend Agent")
    response = trend_agent.invoke({"messages": HumanMessage(f"Analyze {state['query']}")})
    return {"results": [f"QUANT ANALYSIS:\n{response['output']}"]}

def run_noise_agent(state: AgentState):
    """Invokes the Research Agent"""
    print("Executing Noise Agent")
    response = noise_agent.invoke({"messages": HumanMessage(f"Find news for {state['query']}")})
    return {"results": [f"RESEARCH ANALYSIS:\n{response['output']}"]}

def aggregator(state: AgentState):
    """Combines results into a final answer"""
    print("Aggregating Results")
    final_prompt = (
        f"Combine the following reports into a comprehensive investment memo for {state['query']}:\n\n"
        + "\n\n".join(state["results"])
    )
    response = model.invoke({"messages" : HumanMessage(final_prompt)})
    print(f"\nFINAL ANSWER:\n{response.content}")
    return {"results": [response.content]}

class RouteSchema(BaseModel):
    targets: List[Literal["quant", "research"]] = Field(
        description="Which agents to hire? Quant for numbers/charts, Research for news/sentiment."
    )

workflow = StateGraph(AgentState)


workflow.add_node("trend_node", run_trend_agent)
workflow.add_node("noise_node", run_noise_agent)
workflow.add_node("aggregator", aggregator)


def route_query(state: AgentState) -> List[Send]:
    structured_llm = model.with_structured_output(RouteSchema)
    decision = structured_llm.invoke(f"Analyze: {state['query']}")
    
    routes = []
    if "quant" in decision.targets:
        routes.append(Send("trend_node", state))
    if "research" in decision.targets:
        routes.append(Send("noise_node", state))
    if not routes:
        routes = [Send("trend_node", state), Send("noise_node", state)]
        
    return routes

workflow.add_conditional_edges(START, route_query)
workflow.add_edge("trend_node", "aggregator")
workflow.add_edge("noise_node", "aggregator")
workflow.add_edge("aggregator", END)

app = workflow.compile()



if __name__ == "__main__":
    user_query = "What is the outlook for AAPL stock?"
    inputs = {"query": HumanMessage(user_query), "results": []}

    for output in app.stream(inputs):
        pass 
